<div style="padding:20px; 
            color:#150d0a;
            margin:10px;
            font-size:220%;
            text-align:center;
            display:fill;
            border-radius:20px;
            border-width: 5px;
            border-style: solid;
            border-color: #150d0a;
            background-color:#1ee353;
            overflow:hidden;
            font-weight:500"> RAG - 智能家居LLM</div>

<div style="text-align: center;">
    <img src="./src/llm.jpg" alt="Example Image" width="800" height="500">
</div>


  <div style="padding:20px; 
              color:blue;
              margin:10px;
              font-size:150%;
              text-align:center;
              display:fill;
              border-radius:20px;
              border-width: 5px;
              background-color:#eca912;
              overflow:hidden;
              font-weight:500">
    <b>介绍</b>
  </div>

这个笔记本包含一个使用 LangChain 框架、FAISS 作为向量数据库以及 RAGAS 作为评估指标的完整问答管道。

构建 RAG 管道的主要步骤可以总结如下（在文本清理后进行基本的 RAG 管道）：

**模型定义**：模型类定义

**数据摄取**：从网站加载数据

**实例化**：模型 llama2-7b 或 falcon-7b-v2

**索引**：使用 RecursiveCharacterTextSplitter 进行分块索引

**嵌入**：使用 HuggingFaceBgeEmbeddings BAAI/bge-large-en-v1.5

**QA 链检索**：使用 HuggingFacePipeline 和 RetrievalQA

**评分**：选择前 k 个最相似的结果

**评估**：使用 RAGAS 的 TestsetGenerator 进行评估，包括忠实度、答案相关性、上下文召回率、上下文精确率和答案正确性。


  <div style="padding:20px; 
              color:blue;
              margin:10px;
              font-size:150%;
              text-align:center;
              display:fill;
              border-radius:20px;
              border-width: 5px;
              background-color:#eca912;
              overflow:hidden;
              font-weight:500">
    <b>相关描述介绍</b>
  </div>

  <div style="padding:20px; 
              color:blue;
              margin:10px;
              font-size:150%;
              text-align:center;
              display:fill;
              border-radius:20px;
              border-width: 5px;
              background-color:#eca912;
              overflow:hidden;
              font-weight:500">
    <b>目录</b>
  </div>

* [1. 依赖](#dep)

   * [1.1. 安装](#ins)

   * [1.2. 包的导入](#lib)

   * [1.3. 设备配置](#drv)

   * [1.4. API key设置](#key)
   
* [2. 模型定义](#mod)  

* [3. FAISS RAG Pipeline](#faiss)

   * [3.1. 数据加载](#get)        

   * [3.2. 索引](#ind)  

      * [3.2.1. 实例化模型](#ins)

      * [3.2.2. 分割](#split)

      * [3.2.3. 嵌入](#emb)

      * [3.2.4. 向量存储](#vec)

   * [3.3. QA 链检索](#qa)

* [4. 模型评估](#eval)

   * [4.1. 相似度](#sim)

* [5.  LLM智能家居指令控制模型](#steps)

   * [5.1.数据加载](#trainllm1)

     
   * [5.2. 模型微调](#trainllm2)

   * [5.3. 模型部署](#trainllm3)
      

* [6. 智能家居一键式配置](#steps6)


   * [6.1. 结构化数据加载](#61)

     
   * [6.2. 调用大模型进行关键配置提取](#62)

   * [6.3. 提取结果展示](#63)

* [7. 实时天气数据获取](#steps7)


   * [7.1. 结构化数据加载](#71)

     
   * [7.2. 调用大模型预测分析](#72)


# <font color='289C4E'>1. 依赖 📚<font><a class='anchor' id='dep'></a>

## <font color='289C4E'>1.1.  安装 📑<font><a class='anchor' id='ins'></a>

In [6]:
!pip install -U -qq tiktoken cohere openai

In [7]:
!pip install -U -qq torch==2.1.2 torchvision torchaudio torchtext

In [8]:
!pip install -U -qq langchain pypdf faiss-gpu  
!pip install -U -qq transformers InstructorEmbedding sentence_transformers  
!pip install -U -qq huggingface-hub accelerate bitsandbytes xformers einops

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.2 requires torch==2.1.2, but you have torch 2.3.0 which is incompatible.
torchtext 0.16.2 requires torch==2.1.2, but you have torch 2.3.0 which is incompatible.
torchvision 0.16.2 requires torch==2.1.2, but you have torch 2.3.0 which is incompatible.


In [ ]:
!pip install -U -qq ragas

## <font color='289C4E'>1.2. 包的导入 📑<font><a class='anchor' id='lib'></a>

In [32]:
# Global
import os
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["CURL_CA_BUNDLE"]=""
import pandas as pd
import torch
import pickle
import matplotlib.pyplot as plt
from datasets import Dataset
os.environ['REQUESTS_CA_BUNDLE'] = ''
# Transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
from datasets import DatasetDict, Dataset
import json


# Langchain
from langchain_community.document_loaders import WebBaseLoader
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain.callbacks import StdOutCallbackHandler
import os
from langchain_wenxin.chat_models import ChatWenxin

from langchain_community.chat_models import QianfanChatEndpoint
from langchain_core.language_models.chat_models import HumanMessage
from langchain_community.llms import QianfanLLMEndpoint
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.docstore.document import Document
import requests
import os
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.text_splitter import CharacterTextSplitter
import pickle
import os
import pdb
import requests
from bs4 import BeautifulSoup
# Ragas
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from ragas import evaluate

# Warnings
import warnings
warnings.filterwarnings("ignore")

## <font color='289C4E'>1.3. 设备配置 📖<font><a class='anchor' id='drv'></a>

In [33]:
import torch
# 查看可用的设备
print("CUDA available: ", torch.cuda.is_available())
# 指定使用GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CUDA available:  False


## <font color='289C4E'>1.4. API Keys设置 🗝️<font><a class='anchor' id='keys'></a>

In [34]:
os.environ["QIANFAN_AK"] = "cmpZpRZWsuSaO8uuvrPiAeId"
os.environ["QIANFAN_SK"] = "FmHEECYgpyFLjKqyG09xk9WaoTQ4AZWo"

# <font color='289C4E'>2. 模型的定义 🧹<font><a class='anchor' id='mod'></a>

这种管道的方法是创建具有所有参数的模型函数，这将允许实例化具有其超参数的对应大模型

In [35]:
class Model:
    def __init__(self, model_name ='wenxin', temperature = 0,
                 top_p = 0.95, repetition_penalty = 1.15,
                 split_chunk_size = 512, split_overlap = 16, k = 3,
                 embeddings_path = "./faiss_index_hp",
                 persist_directory = "./faiss-vectordb"):

        """
        Initialize the Model class with default values and parameters.
        :param model_name: Name of the model (default: 'wenxin')
        :param temperature: Controls the randomness of the predictions (default: 0)
        :param top_p: Top p value for nucleus sampling (default: 0.95)
        :param repetition_penalty: Penalty for repeated tokens (default: 1.15)
        :param split_chunk_size: Size of chunks to split the input text (default: 800)
        :param split_overlap: Overlap between split chunks (default: 0)
        :param k: Value for some parameter 'k' (default: 3)
        """
        # 参数
        self.model_name = model_name
        self.temperature = temperature
        self.top_p = top_p
        self.repetition_penalty = repetition_penalty
        self.split_chunk_size = split_chunk_size
        self.split_overlap = split_overlap
        self.embeddings_model_repo = 'BAAI/bge-large-en-v1.5'
        self.k = k
        self.embeddings_path = embeddings_path
        self.persist_directory = persist_directory

自定义一个pipeline的一个包装类

In [36]:
class CustomQianfanPipeline:
    def __init__(self, streaming=True):
        self.llm = QianfanLLMEndpoint(streaming=streaming)
        self.task = "text-generation"  # 添加任务属性

    def __call__(self, inputs):
        if isinstance(inputs, list):
            prompt = inputs[0]
        else:
            prompt = inputs
        response = self.llm(prompt)
        return [{"generated_text": response}]  # 确保返回值是一个包含生成文本的字典

In [37]:
def get_model(config):
    # Extract the model name from the configuration
    model_name = config.model_name
    # Dictionary containing model parameters
    model_params = {
        'wenxin': {
            'repo': 'daryl149/llama-2-7b-chat-hf',
            'max_len': 2048,
            'extra_params': {
                'device_map': 'auto',
                'torch_dtype': torch.float16,
                'low_cpu_mem_usage': True,
                'trust_remote_code': True
            }
        },
        'falcon': {
            'repo': 'h2oai/h2ogpt-gm-oasst1-en-2048-falcon-7b-v2',
            'max_len': 1024,
            'extra_params': {
                'device_map': 'auto',
                'torch_dtype': torch.float16,
                'low_cpu_mem_usage': True,
                'trust_remote_code': True
            }
        },
    }

    # Check if the provided model name exists in the predefined model parameters
    if model_name in model_params:
        # Get the parameters for the specified model
        params = model_params[model_name]
        model_repo = params['repo']

        # Extract any additional parameters specific to the model
        extra_params = params.get('extra_params', {})

        # Instantiate a tokenizer based on the model repository and extra parameters
        tokenizer = AutoTokenizer.from_pretrained(model_repo, **extra_params)

        model= CustomQianfanPipeline(streaming=True)
        # Get the maximum length for the model
        max_len = params['max_len']
    else:
        # Raise an error if the provided model is not implemented in the dictionary
        raise ValueError("The provided model is not implemented.")

    # Return the instantiated tokenizer, model, and maximum length
    return tokenizer, model, max_len

# <font color='289C4E'>3. FAISS RAG Pipeline 📳<font><a class='anchor' id='faiss'></a>

<img src="src/output.gif" alt="Animated GIF">

## <font color='289C4E'>3.1. 数据加载 🎡<font><a class='anchor' id='get'></a>

第一步是获取我们的数据。我们将使用 WebBaseLoader，并选择要加载的网站。

In [41]:
# Load the data
loader = WebBaseLoader(
    "http://www.sohu.com/a/681843047_121733165"
)
documents = loader.load()

In [42]:
# Save the loaded documents using pickle

with open('/loaded_documents.pkl', 'wb') as file:
    pickle.dump(documents, file)


In [43]:
# Load the saved documents

with open('/loaded_documents.pkl', 'rb') as file:
    documents = pickle.load(file)


## <font color='289C4E'>3.2. 索引 🌊<font><a class='anchor' id='ind'></a>

为了创建索引，需要遵循以下几个步骤：

- 选择一种分割方法及其超参数：我们将使用**RecursiveCharacterTextSplitter**。该方法递归地尝试通过不同的字符进行分割，以找到合适的分割点。

- 选择嵌入模型：在我们的案例中，选择**BAAI/bge-large-en-v1.5**。

- 选择一个向量存储：**FAISS**。


### <font color='289C4E'>3.2.1. 实例化模型 🔠<font><a class='anchor' id='ins'></a>

We have to instatiate the model in order to pass the corresponding hyperparameters.

In [39]:
# Creating an instance of the Model Class and get the model
my_faiss_model = Model(model_name="wenxin",
                       embeddings_path = "/faiss_index_hp",
                       persist_directory = "./faiss-vectordb")
   
tokenizer, model_faiss, max_len= get_model(config=my_faiss_model)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


### <font color='289C4E'>3.2.2 分割 🪓<font><a class='anchor' id='split'></a>

In [44]:
# Instantiate a RecursiveCharacterTextSplitter using parameters from 'my_model'

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=my_faiss_model.split_chunk_size,  # Set the size of each text chunk
    chunk_overlap=my_faiss_model.split_overlap   # Set the overlap between consecutive chunks
)

# Split the documents into smaller texts using the text splitter
splitted_text = text_splitter.split_documents(documents)

# Print the number of resulting text chunks
print(f'Number of Chunks: {len(splitted_text)}')

# Print the number of original pages/documents
print(f'Number of Pages: {len(documents)}')

Number of Chunks: 4
Number of Pages: 1


### <font color='289C4E'>3.2.3. 嵌入 🪟<font><a class='anchor' id='emb'></a>

In [45]:
# Download embeddings model

embeddings = HuggingFaceBgeEmbeddings(
    model_name = my_faiss_model.embeddings_model_repo,
    model_kwargs = {"device": "cpu"},
    encode_kwargs = {"normalize_embeddings": True}
)

### <font color='289C4E'>3.2.4. 向量存储 🗂️<font><a class='anchor' id='vec'></a>

In [46]:
# Create the vector store

faiss_vectordb = FAISS.from_documents(
                documents = splitted_text,
                embedding = embeddings
                )

## <font color='289C4E'>3.3. QA链检索 🚒<font><a class='anchor' id='qa'></a>

<div style="text-align: center;">
    <img src="./src/llm2.jpg" alt="Example Image" width="800" height="500">
</div>

下一步，我们需要创建我们的检索管道，为了将提示与额外的上下文结合起来，我们需要准备一个提示模板。可以从下面显示的提示模板轻松定制提示。该管道将使用**HuggingFacePipeline**创建。

In [47]:
llm = HuggingFacePipeline(pipeline=model_faiss)

# Corvert the vector store in a retriever
retriever = faiss_vectordb.as_retriever(search_type="similarity", search_kwargs={"k": my_faiss_model.k})
# Set prompt template
template = """你是一位智能家居助手，可以帮助我控制家居生活并回答问题。
根据检索的上下文来回答问题，如果不知道答案，只需说明不知道。

Question: {question}
Context: {context}

Answer:
"""

prompt = ChatPromptTemplate.from_template(template)


In [75]:
# Set the question
question = "请帮我提供家居健康智能配置"
# Callback: helps to show and manage the output
handler =  StdOutCallbackHandler()

# Initializing a combined QA chain
qa_combined_chain = RetrievalQA.from_chain_type(
                    llm=llm,
                    chain_type_kwargs = {"prompt": prompt},
                    retriever=retriever,
                    callbacks=[handler],
                    return_source_documents=True)

# Initialize the query
response = qa_combined_chain({"query": question, "context": retriever})

# Print response
response["result"]



> Entering new RetrievalQA chain...

> Finished chain.


'智能家居健康配置包括健康家居监测设备和智能家电。智能家电包括智能空调、智能冰箱、智能洗衣机、智能灯光等，能够通过智能化技术提高居住舒适度，提供更贴心的服务。健康家居监测设备包括智能体重秤、智能手环、智能血压计等，能够监测身体健康状况，提供健康建议和预警。\n\n您可以根据您的需求选择合适的智能家居控制系统和智能家电，以实现家居健康智能化配置。同时，您也可以关注一些智能家居品牌和平台，了解更多有关智能家居健康配置的信息。'

In [76]:
response["source_documents"]

[Document(metadata={'source': 'http://www.sohu.com/a/681843047_121733165', 'title': '你应该了解的智能家居控制系统_智能化_设备_用户', 'description': '\n 华为智能家居控制系统：华为公司推出的智能家居控制系统，能够实现对智能家电、照明、安防等设备的远程控制和管理，支持多种语音助手和智能设备，具有高度的安全性和可靠性。 京东智能家居控制系统：京东公司推出…', 'language': 'No language found.'}, page_content='你应该了解的智能家居控制系统_智能化_设备_用户\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n新闻\n体育\n汽车\n房产\n旅游\n教育\n时尚\n科技\n财经\n娱乐\n更多\n\n母婴\n健康\n历史\n军事\n美食\n文化\n星座\n专题\n游戏\n搞笑\n动漫\n宠物\n\n\n\n\n\n\n无障碍\n\n关怀版\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n爱科技er\n\n\n\n文章\n总阅读\n\n\n\n\n\n\n查看TA的文章>\n\n \n\n评论\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                    你应该了解的智能家居控制系统                \n\n\n\n\n                        2023-06-03 22:25        \n发布于：北京市\n\n\n\n\n智能家居控制系统是一种智能化的家居系统，通过集成各种智能设备和互联网技术，实现对家居环境的远程控制和智能化管理。它能够提高生活质量，提升居住舒适度，提高居家安全性和便利性，成为未来智慧家居的重要组成部分。\n\n智能家居控制系统包括家庭安防、照明系统、智能家电、温控系统、音视频系统等多个领域。以下是几个相对成熟的智能家居控制系统。'),
 Document(metadata={'source': 'http://www.sohu.com/a/681843047_121733165', 'title': '你应该了解的智能家居控制系统_智能化

# <font color='289C4E'>4. 模型评估 ❓<font><a class='anchor' id='eval'></a>

## <font color='289C4E'>4.1. 相似度 🤼‍♂️<font><a class='anchor' id='sim'></a>

FAISS有一些特定的方法用于检查相似度。其中之一是 **similarity_search_with_score**（搜索与查询向量最相似的前k个向量），它不仅可以返回文档，还可以返回查询向量与这些文档之间的距离分数。返回的距离分数是L2距离。因此，较接近0的较低分数表示相似度更好。

In [77]:
### Testing Similarity Search

# Set the question
question = "请帮我提供家居健康智能配置"

results_with_scores = faiss_vectordb.similarity_search_with_score(question, k = my_faiss_model.k)

for doc, score in results_with_scores:
    metadata = doc.metadata if hasattr(doc, 'metadata') else None
    source = metadata.get('source') if metadata else None
    # title = metadata.get('title') if metadata else None

    print(f"Source: {source}, Score: {score}")

Source: http://www.sohu.com/a/681843047_121733165, Score: 0.4059630036354065
Source: http://www.sohu.com/a/681843047_121733165, Score: 0.41701382398605347
Source: http://www.sohu.com/a/681843047_121733165, Score: 0.5442841053009033


In [78]:
results_with_scores

[(Document(metadata={'source': 'http://www.sohu.com/a/681843047_121733165', 'title': '你应该了解的智能家居控制系统_智能化_设备_用户', 'description': '\n 华为智能家居控制系统：华为公司推出的智能家居控制系统，能够实现对智能家电、照明、安防等设备的远程控制和管理，支持多种语音助手和智能设备，具有高度的安全性和可靠性。 京东智能家居控制系统：京东公司推出…', 'language': 'No language found.'}, page_content='你应该了解的智能家居控制系统_智能化_设备_用户\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n新闻\n体育\n汽车\n房产\n旅游\n教育\n时尚\n科技\n财经\n娱乐\n更多\n\n母婴\n健康\n历史\n军事\n美食\n文化\n星座\n专题\n游戏\n搞笑\n动漫\n宠物\n\n\n\n\n\n\n无障碍\n\n关怀版\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n爱科技er\n\n\n\n文章\n总阅读\n\n\n\n\n\n\n查看TA的文章>\n\n \n\n评论\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                    你应该了解的智能家居控制系统                \n\n\n\n\n                        2023-06-03 22:25        \n发布于：北京市\n\n\n\n\n智能家居控制系统是一种智能化的家居系统，通过集成各种智能设备和互联网技术，实现对家居环境的远程控制和智能化管理。它能够提高生活质量，提升居住舒适度，提高居家安全性和便利性，成为未来智慧家居的重要组成部分。\n\n智能家居控制系统包括家庭安防、照明系统、智能家电、温控系统、音视频系统等多个领域。以下是几个相对成熟的智能家居控制系统。'),
  0.405963),
 (Document(metadata={'source': 'http://www.sohu.com/a/681843047_121733165', 'title': '你应该

另一个指标是 **similarity_search_with_relevance_scores**。该指标的范围为0到1。因此，接近1的较高分数表示相似度更好。

In [79]:
question = "请帮我提供家居健康智能配置"

results_with_scores = faiss_vectordb.similarity_search_with_relevance_scores(question, k = my_faiss_model.k)


for doc, score in results_with_scores:
    metadata = doc.metadata if hasattr(doc, 'metadata') else None
    source = metadata.get('source') if metadata else None
    # title = metadata.get('title') if metadata else None

    print(f"Source: {source}, Score: {score}")



Source: http://www.sohu.com/a/681843047_121733165, Score: 0.712940807218545
Source: http://www.sohu.com/a/681843047_121733165, Score: 0.7051266972109282
Source: http://www.sohu.com/a/681843047_121733165, Score: 0.6151330182496784


# <font color='289C4E'>5. LLM智能家居指令控制模型 📈<font><a class='anchor' id='steps'></a>

## <font color='289C4E'>5.1. 数据加载 🤼‍♂️<font><a class='anchor' id='trainllm1'></a>

In [4]:
dataset_file="D:\\Desktop\\rag+langchain\\src\\dataset_info.json"

def load_json_dataset(json_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data
ds=load_json_dataset(dataset_file)
processed_data = {
    'instruction': [item['instruction'] for item in ds],
    'input': [item['input'] for item in ds],
    'output': [item['output'] for item in ds]
}
ds= Dataset.from_dict(processed_data)


## <font color='289C4E'>5.2. 模型微调 🤼‍♂️<font><a class='anchor' id='trainllm2'></a>

<div style="text-align: center;">
    <img src="./src/lora.png" alt="Example Image" width="800" height="500">
</div>

### PEFT（Parameter-Efficient Fine-Tuning）

**基本原理**:
- **参数高效微调**：PEFT 是一种参数高效的微调方法，旨在通过调整模型的部分参数而非全部参数来实现对新任务的适应。PEFT 的目标是最大化微调的效率和效果，同时最小化所需的计算资源和存储空间。
- **常见方法**:
  - **LoRA**：通过低秩分解来减少需要更新的参数。
  - **Adapter**：在模型的特定层之间插入小型适配器模块，仅微调这些模块的参数。
  - **Prefix Tuning**：对输入序列的前缀部分进行参数微调，使得模型能够适应新的任务。

### LoRA 微调

**基本原理**:
- **低秩分解**：LoRA 微调通过将预训练模型的权重矩阵 \(W\) 分解为两个低秩矩阵 \(A\) 和 \(B\)，即 \(W = W_0 + \Delta W\)，其中 \(\Delta W = A \cdot B\)。其中，\(W_0\) 是预训练模型的原始权重矩阵，\(\Delta W\) 是需要学习的微调部分。通过选择合适的低秩矩阵 \(A\) 和 \(B\)，可以大幅减少微调参数的数量。
- **训练过程**：在微调过程中，仅更新低秩矩阵 \(A\) 和 \(B\)，而保持原始权重矩阵 \(W_0\) 不变。这种方法能够在减少计算和存储开销的情况下，实现对模型的有效微调。

### 实现方式

1. **LoRA**:
   - 将原始权重矩阵 \(W\) 表示为 \(W_0 + \Delta W\)，其中 \(\Delta W = A \cdot B\)。
   - 在微调过程中，保持 \(W_0\) 不变，仅更新 \(A\) 和 \(B\) 以适应新任务。


In [5]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh", low_cpu_mem_usage=True)

In [6]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(task_type=TaskType.CAUSAL_LM, target_modules=".*\.1.*query_key_value", modules_to_save=["word_embeddings"])

In [7]:
model = get_peft_model(model, config)

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


In [15]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=10
)

In [16]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [17]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Map:   0%|          | 0/129 [00:00<?, ? examples/s]

Map: 100%|██████████| 129/129 [00:00<00:00, 3071.42 examples/s]


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 129
})

In [18]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [19]:
trainer.train()

  6%|▋         | 10/160 [01:53<28:04, 11.23s/it]

{'loss': 2.4742, 'grad_norm': 3.694650888442993, 'learning_rate': 4.6875e-05, 'epoch': 0.62}


 12%|█▎        | 20/160 [03:46<25:55, 11.11s/it]

{'loss': 2.145, 'grad_norm': 3.5705406665802, 'learning_rate': 4.375e-05, 'epoch': 1.24}


 19%|█▉        | 30/160 [05:41<25:05, 11.58s/it]

...

100%|██████████| 160/160 [30:21<00:00, 11.29s/it]

{'loss': 0.3224, 'grad_norm': 1.638658881187439, 'learning_rate': 0.0, 'epoch': 9.92}


100%|██████████| 160/160 [30:22<00:00, 11.39s/it]

{'train_runtime': 1822.9843, 'train_samples_per_second': 0.708, 'train_steps_per_second': 0.088, 'train_loss': 0.9243620336055756, 'epoch': 9.92}


TrainOutput(global_step=160, training_loss=0.9243620336055756, metrics={'train_runtime': 1822.9843, 'train_samples_per_second': 0.708, 'train_steps_per_second': 0.088, 'total_flos': 777462212395008.0, 'train_loss': 0.9243620336055756, 'epoch': 9.922480620155039})

## <font color='289C4E'>5.3. 集成部署 🤼‍♂️<font><a class='anchor' id='trainllm3'></a>

In [21]:
model = model
ipt = tokenizer("Human: {}\n{}".format("开启美的空调", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
tokenizer.decode(model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True)

'Human: 开启美的空调\n\nAssistant: 开启美的空调\nThe Assistant has to wait until the next power outage, or may not switch on if the power is back on.'

# <font color='289C4E'>6. LLM智能家居一键式配置 📈<font><a class='anchor' id='steps6'></a>

<div style="text-align: center;">
    <img src="./src/llm4.png" alt="Example Image" width="800" height="500">
</div>

流程设计
PDF 文件读取：

使用 Python 库（如 PyPDF2、pdfplumber）读取 PDF 文件内容，并将其转换为纯文本格式。
文本分割：

将转换后的文本按段落或页面进行分割，以便逐步处理长文档内容。
使用 LangChain 的 RecursiveCharacterTextSplitter 进行文本分割。
嵌入生成：

使用预训练的嵌入模型（如 BAAI/bge-large-en-v1.5）将分割后的文本转换为向量表示。
向量存储：

将生成的文本向量存储在向量数据库中（如 FAISS），以便后续的快速查询和相似度计算。
内容抽取：

根据查询需求，使用向量存储库中存储的文本向量进行相似度搜索，提取与查询相关的文本内容。
利用 LangChain 的 LLM 模型进行自然语言处理，进一步提取和生成所需的信息。

## <font color='289C4E'>6.1. 结构化数据加载 🤼‍♂️<font><a class='anchor' id='61'></a>

In [53]:
source_path="D:\\Desktop\\rag+langchain\\pdf\\"
def get_pdf_data(file_path, num_pages = 1):
  reader = PdfReader(source_path+file_path)
  full_doc_text = ""
  for page in range(len(reader.pages)):
    current_page = reader.pages[page]
    text = current_page.extract_text()
    full_doc_text += text


  return Document(
        page_content=full_doc_text,
        metadata = {"source": file_path}
    )

In [54]:
def source_docs():
    return [get_pdf_data(file) for file in os.listdir("D:\\Desktop\\rag+langchain\\pdf")]

In [57]:

def search_index(source_docs):
    source_chunks = []
    splitter = CharacterTextSplitter(separator=" ", chunk_size=1024, chunk_overlap=0)

    for source in source_docs:
        for chunk in splitter.split_text(source.page_content):
            source_chunks.append(Document(page_content=chunk, metadata=source.metadata))
    with open("search_index.pickle", "wb") as f:
        pickle.dump(FAISS.from_documents(source_chunks, embeddings), f)

## <font color='289C4E'>6.2. 调用大模型进行关键配置提取 🤼‍♂️<font><a class='anchor' id='62'></a>

<div style="text-align: center;">
    <img src="./src/llm3.png" alt="Example Image" width="800" height="800">
</div>

In [58]:
chain = load_qa_with_sources_chain(llm,verbose=False, chain_type="stuff")
def print_answer(question):
    with open("search_index.pickle", "rb") as f:
        search_index = pickle.load(f)
    print(
        chain(
            {
                "input_documents": search_index.similarity_search(question, k=3),
                "question": question,
            },
            return_only_outputs=True,
        )["output_text"]

    )


In [61]:
sources = source_docs()
search_index(sources)

## <font color='289C4E'>6.3.抽取结果展示 🤼‍♂️<font><a class='anchor' id='63'></a>

In [65]:
try:
  print_answer("question=家居常见配置") # your question here
except Exception as e:
  print(f"Unexpected error: {e}")

家居常见配置主要包括空调、加湿器、照明、安防系统等智能家居设备，以下是常见配置参数：

* 空调：温度设置范围为夏季的24°C至冬季的20°C，模式包括制冷、制热、除湿、送风，风速分为自动、低、中、高，风向可调节，具有定时开关功能。
* 加湿器：湿度设置范围为40%-60%（建议舒适湿度），模式包括低、中、高、自动，水箱容量从2L到6L不等，具有定时开关功能和调节雾量大小的功能。
* 照明：亮度可从0%-100%调节，色温从2700K的暖光到6500K的冷光调节，具有阅读模式、休闲模式、夜灯模式等不同的模式，并可实现定时开关功能。同时，智能照明设备通常支持语音控制和手机APP控制。
* 安防系统：包括摄像头、报警系统和存储功能。摄像头分辨率可达1080P或4K，具备红外夜视功能和运动检测报警功能。报警系统可以本地存储或云存储，并支持手机APP远程监控。
* 其他设备：包括智能门锁和智能音箱等。智能门锁支持指纹解锁、密码解锁、卡片解锁和手机APP解锁等多种方式，安全等级从C级到A级不等，并具有防撬报警和低电量报警等功能。智能音箱支持主流的语音助手，如Amazon Alexa、Google Assistant和Apple Siri，具有高保真音质和360°全向音频，并支持Wi-Fi和蓝牙连接。

以上设备是常见的智能家居配置，并没有具体提到Michael Jackson相关的配置信息。

SOURCES: main.pdf


# <font color='289C4E'>7. 实时天气数据获取 📈<font><a class='anchor' id='steps7'></a>

## <font color='289C4E'>7.1.网页实时数据提取 🤼‍♂️<font><a class='anchor' id='71'></a>

In [73]:
def get_weather_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 这里以某个具体网站的天气信息为例
    weather = soup.find('div', class_='c-city-weather-forecast')
    temperature = weather.find('span', class_='temp').text
    temperature_last = weather.find('span', class_='temp last').text

    return temperature, temperature_last

weather_url = 'https://www.qweather.com/weather/beijing-101010100.html'
temperature, weather = get_weather_data(weather_url)
print(f"Temperature: {temperature}, Temperature_last: {weather}")

Temperature: 34°, Temperature_last: 24°


## <font color='289C4E'>7.2.调用大模型预测分析 🤼‍♂️<font><a class='anchor' id='72'></a>

In [75]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


# 定义LangChain
prompt_template = PromptTemplate(template="当前的天气温度是 {temperature} 最终的天气温度是 {weather}. 生成详细的天气报告预测的.", input_variables=["temperature", "weather"])

chain = LLMChain(llm=llm, prompt=prompt_template)

# 生成天气报告
def generate_weather_report(temperature,  weather):
    weather_report = chain.run({"temperature": temperature, "weather":  weather})
    return weather_report

weather_report = generate_weather_report(temperature, weather)
print(weather_report)


根据当前温度和最终温度的差异，我为您生成了一份详细的天气报告预测：

天气概述：
根据当前的温度数据，预计未来的天气将会出现明显的降温。当前的气温为34°，而最终的气温预计为24°。这种气温下降可能会带来一些不同的天气变化，例如可能会影响人们的户外活动和计划。

天气变化分析：

* 风力：如果气温下降，风力可能会增强。请注意保护您的财物，避免在强风中行走或骑车。
* 湿度：湿度的变化可能会影响您的呼吸和健康。如果您有呼吸问题或过敏症状，请注意保持室内空气流通。
* 衣物建议：由于气温下降，建议您携带一些保暖衣物，如厚外套、毛衣和帽子。同时，保持水分摄入，以应对可能的干燥天气。
* 防晒措施：虽然气温下降，但请注意避免过度暴露在阳光下。防晒霜、遮阳帽和太阳眼镜等防晒用品是必要的。

总结：
根据当前气温和最终气温的差异，我建议您关注天气预报以了解未来的天气变化。根据预测，风力可能会增强，湿度可能会有所变化。请注意保护您的财物，并携带适当的衣物和防晒用品。同时，保持水分摄入以应对可能的干燥天气。希望这些信息对您有所帮助！
